In [1]:
import requests
from bs4 import BeautifulSoup

class RequestCustom:
    def __init__(self, url, site_name):
        self.url = url
        self.site_name = site_name

    def get_url(self):
        return self.url

    def get_request(self):
        url=self.get_url()     
        return requests.get(url, allow_redirects=False)

    def get_status(self):
        request = self.get_request()
        status = request.status_code
        return status

    def get_soup(self):
        request=self.get_request()
        status = self.get_status()
        if status == 200:
            soup = BeautifulSoup(request.text, 'html.parser')
            return soup
        else:
            print("Error: ", status)
            return None

In [2]:
import csv

class CsvCustom:

    def __init__(self, site_name):
        self.site_name = site_name

    def create_csv(self):
        file = open(f'{self.site_name}.csv', 'w',newline='',encoding='utf-8-sig')
        writer = csv.writer(file)
        headers = ["title", "link", "description", "price"]
        writer.writerow(headers)

    def write_csv(self,**kwargs): 
        title = kwargs['title']
        link = kwargs['link']
        description = kwargs['description']
        price = kwargs['price']

        file = open(f'{self.site_name}.csv', 'a', newline='', encoding='utf-8-sig')
        writer = csv.writer(file)
        contents = ([title,link, description, price])
        writer.writerow(contents)
        return file

    def close_csv(self,file):
        file.close()

In [3]:
number=1

site_energy_star = RequestCustom(url=f"https://www.energystar.gov/productfinder/product/certified-residential-refrigerators/?formId=077786-589-4-8882-821429358&scrollTo=3719&search_text=&low_price=&high_price=&price_filter=Under+%24500&price_filter=%24500-%24749&price_filter=%24750-%24999&price_filter=%241000-%241499&price_filter=%241500-%241999&price_filter=%242000-%242999&price_filter=%243000+and+up&price_filter=Price+not+available&is_most_efficient_filter=0&height_in_isopen=0&width_in_isopen=0&brand_name_isopen=0&markets_filter=United+States&markets_filter=Canada&zip_code_filter=&product_types=Select+a+Product+Category&sort_by=percent_less_energy_use_than_us_federal_standard&sort_direction=desc&currentZipCode=93300&page_number={number}", site_name="energy_star")
status = site_energy_star.get_status()

csv_energystar=CsvCustom("energystar")
csv_energystar.create_csv()

while status == 200:
    #listes_prods
    soup = site_energy_star.get_soup()

    listes_prods=soup.find_all("div", attrs={"class":"row certified-residential-refrigerators"})

    for prods in listes_prods:
        title = prods.find("div", attrs={"class":"title"}).get_text()
        link ="https://www.energystar.gov"+prods.find("div",attrs={"class":"title"}).find("a")['href']
        description = prods.find("div", attrs={"class":"value"}).get_text()
        price_element = prods.find("a", attrs={"class": "priceRange"})
        price = price_element.get_text() if price_element else ""

        price = ''.join(price.split())
        title = title.strip()
        description = description.strip()
        
        file=csv_energystar.write_csv(title=title, link=link, description=description, price=price)
    number=number+1
    site_energy_star = RequestCustom(url=f"https://www.energystar.gov/productfinder/product/certified-residential-refrigerators/?formId=077786-589-4-8882-821429358&scrollTo=3719&search_text=&low_price=&high_price=&price_filter=Under+%24500&price_filter=%24500-%24749&price_filter=%24750-%24999&price_filter=%241000-%241499&price_filter=%241500-%241999&price_filter=%242000-%242999&price_filter=%243000+and+up&price_filter=Price+not+available&is_most_efficient_filter=0&height_in_isopen=0&width_in_isopen=0&brand_name_isopen=0&markets_filter=United+States&markets_filter=Canada&zip_code_filter=&product_types=Select+a+Product+Category&sort_by=percent_less_energy_use_than_us_federal_standard&sort_direction=desc&currentZipCode=93300&page_number={number}", site_name="energy_star")
    status = site_energy_star.get_status()
    
    csv_energystar.close_csv(file)  

if status == 404:
    print(f"The page {number} doesn't exist")

KeyboardInterrupt: 